## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [7]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.44 ms ± 97 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 34.3 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 27 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


### Ejercicio 3.2 apartado A

In [10]:
import numpy as np
from numba import njit, jit
import cupy as cp
from cupyx.profiler import benchmark
# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

def cupy_copia():
    # Copiar arrays de CPU a GPU
    a_gpu = cp.asarray(a_cpu)
    b_gpu = cp.asarray(b_cpu)
    
    # Calcular en GPU (usando ufunc de CuPy)
    result_gpu = a * a_gpu**2 + b * b_gpu + c
    
    # Copiar resultado de vuelta a CPU
    return cp.asnumpy(result_gpu)

# Medir tiempo con la función benchmark de CuPy
cp_benchmark = benchmark(cupy_copia, n_repeat=5, n_warmup=2)
c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)
gpu_avg_time = np.average(cp_benchmark.gpu_times) * 1e3
print(f"Resultado benchmark CuPy (con copia):")
print(f"  Tiempo: {gpu_avg_time:.3f} ms")


def cupy_sin_copia():
    # Crear arrays directamente en GPU (equivalente a np.random.rand)
    a_gpu = cp.random.rand(size)
    b_gpu = cp.random.rand(size)
    
    # Calcular en GPU
    result_gpu = a * a_gpu**2 + b * b_gpu + c
    
    # Devolver resultado en GPU (sin copiar a CPU)
    return result_gpu

# Medir tiempo con benchmark de CuPy
cp_benchmark_sincopia = benchmark(cupy_sin_copia, n_repeat=5, n_warmup=2)
gpu_avg_time = np.average(cp_benchmark_sincopia.gpu_times) * 1e3
print(f"Resultado benchmark CuPy (sin copia):")
print(f"  Tiempo: {gpu_avg_time:.3f} ms")
# Evaluating the time
print("Resultados de numba y numpy:")
# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c

float64
Resultado benchmark CuPy (con copia):
  Tiempo: 21.074 ms
Resultado benchmark CuPy (sin copia):
  Tiempo: 5.542 ms
Resultados de numba y numpy:
8.32 ms ± 9.57 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.7 ms ± 43.5 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.6 ms ± 63.7 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


### Ejercicio 3.2 apartado B

In [2]:
import numpy as np
from numba import njit, jit, vectorize, cuda
import cupy as cp
from cupyx.profiler import benchmark
import time
# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

@vectorize(['float64(float64, float64, float64, float64, float64)'], target='cuda')  
def grade2_numba_gpu_elementwise(x, y, a, b, c):
    return a*x*x + b*y + c
# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

# Función para medir incluyendo la copia
def numba_gpu_copia():
    # La copia ocurre automáticamente dentro de la función ufunc
    return grade2_numba_gpu_elementwise(a_cpu, b_cpu, a, b, c)
times = []
for i in range(5):
    start = time.perf_counter()
    result = numba_gpu_copia()
    end = time.perf_counter()
    times.append((end - start) * 1000)  # Convertir a ms

print(f"Tiempo promedio: {np.mean(times):.3f} ms")


# Crear arrays en GPU
a_gpu = cuda.to_device(a_cpu)
b_gpu = cuda.to_device(b_cpu)
# Crear array de salida en GPU
result_gpu = cuda.device_array_like(a_cpu)


@cuda.jit
def grade2_cuda_kernel(x, y, a, b, c, out):
    i = cuda.grid(1)
    if i < x.size:
        out[i] = a * x[i] * x[i] + b * y[i] + c

threads_per_block = 256
blocks_per_grid = (size + (threads_per_block - 1)) // threads_per_block

times_kernel = []
for _ in range(5):
    start = time.perf_counter()
    grade2_cuda_kernel[blocks_per_grid, threads_per_block](a_gpu, b_gpu, a, b, c, result_gpu)
    cuda.synchronize()  # Esperar a que termine la GPU
    end = time.perf_counter()
    times_kernel.append((end - start) * 1000)

print(f"Tiempo CUDA sin copia: {np.mean(times_kernel):.3f} ms")
# Evaluating the time
print("Resultados de numba y numpy:")
# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c

float64
Tiempo promedio: 31.058 ms
Tiempo kernel CUDA: 8.017 ms
Resultados de numba y numpy:
The slowest run took 7.21 times longer than the fastest. This could mean that an intermediate result is being cached.
31.8 ms ± 24 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.6 ms ± 96.2 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.5 ms ± 64 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


### Ejercicio 3.2 apartado C
Al realizar ambos apartados vemos que en ambos casos la ejecucion sin copia es mas rapida que la que copia de cpu a gpu para realizar la operacion; pero la opcion mas rapida es cupy sobre numba Cuda. Siendo en numba cuda de 8 ms y en cupy 5.5ms